In [6]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd

# Fetch stock data
def fetch_stock_data(ticker, period='1y', interval='1d'):
    stock = yf.Ticker(ticker)
    data = stock.history(period=period, interval=interval)
    return data

# Add technical indicators
def add_technical_indicators(data):
    data['SMA_20'] = ta.sma(data['Close'], length=20)
    data['SMA_50'] = ta.sma(data['Close'], length=50)
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['MACD'], data['MACD_Signal'], _ = ta.macd(data['Close'])
    return data

# Calculate moving averages
def calculate_moving_averages(data):
    return {
        "20-Day MA": data['Close'].rolling(window=20).mean().iloc[-1],
        "50-Day MA": data['Close'].rolling(window=50).mean().iloc[-1],
        "200-Day MA": data['Close'].rolling(window=200).mean().iloc[-1]
    }

# Calculate Fibonacci retracement levels
def calculate_fibonacci_levels(data):
    high_range = data['High'].max()
    low_range = data['Low'].min()
    difference = high_range - low_range
    return {
        "0.0% (Swing High)": high_range,
        "23.6%": high_range - 0.236 * difference,
        "38.2%": high_range - 0.382 * difference,
        "50.0%": high_range - 0.5 * difference,
        "61.8%": high_range - 0.618 * difference,
        "100.0% (Swing Low)": low_range
    }

# Calculate support and resistance levels
def calculate_support_resistance(data):
    high_range = data['High'].max()
    low_range = data['Low'].min()

    # Initialize levels
    levels = {"S1": None, "S2": None, "S3": None, "R1": None, "R2": None, "R3": None}

    # Calculate support and resistance
    levels["R1"] = high_range
    levels["S1"] = low_range

    difference = high_range - low_range
    levels["R2"] = high_range - 0.382 * difference
    levels["R3"] = high_range - 0.618 * difference
    levels["S2"] = low_range + 0.382 * difference
    levels["S3"] = low_range + 0.618 * difference

    return levels

# Simple buy/sell recommendation based on SMA and RSI
def simple_recommendation(data):
    last_rsi = data['RSI'].iloc[-1]
    last_price = data['Close'].iloc[-1]
    sma_20 = data['SMA_20'].iloc[-1]
    sma_50 = data['SMA_50'].iloc[-1]

    if last_price > sma_20 and last_price > sma_50 and last_rsi < 70:
        return "Buy"
    elif last_price < sma_50 and last_rsi > 70:
        return "Sell"
    else:
        return "Hold"

# Display results
def display_results(data, ticker, moving_averages, fib_levels, support_resistance, real_time_price):
    print(f"\nResults for {ticker}:")
    print(f"Close Price: {data['Close'].iloc[-1]:.2f}")

    print("\nMoving Averages:")
    for ma, value in moving_averages.items():
        print(f"{ma}: {value:.2f}")

    print("\nFibonacci Retracement Levels:")
    for level, value in fib_levels.items():
        print(f"{level}: {value:.2f}")

    print(f"\nReal-Time Price: {real_time_price:.2f}")

    print("\nSupport Levels:")
    print(f"S1: {support_resistance['S1']:.2f}")
    print(f"S2: {support_resistance['S2']:.2f}")
    print(f"S3: {support_resistance['S3']:.2f}")

    print("\nResistance Levels:")
    print(f"R1: {support_resistance['R1']:.2f}")
    print(f"R2: {support_resistance['R2']:.2f}")
    print(f"R3: {support_resistance['R3']:.2f}")

    # Get buy/sell recommendation
    recommendation = simple_recommendation(data)
    print(f"Recommendation: {recommendation}")

# Main function to run analysis
def run_analysis(tickers):
    for ticker in tickers:
        print(f"\nRunning analysis for {ticker}...")

        # Fetch data
        data = fetch_stock_data(ticker)

        # Check if data is available
        if data.empty:
            print(f"No data available for {ticker}. Please check the ticker symbol.")
            continue

        # Add technical indicators
        data = add_technical_indicators(data)

        # Calculate moving averages
        moving_averages = calculate_moving_averages(data)

        # Calculate Fibonacci retracement levels
        fib_levels = calculate_fibonacci_levels(data)

        # Calculate support and resistance levels
        support_resistance = calculate_support_resistance(data)

        # Fetch real-time price
        real_time_price = yf.Ticker(ticker).history(period="1d")['Close'].iloc[0]

        # Display results
        display_results(data, ticker, moving_averages, fib_levels, support_resistance, real_time_price)

# Input tickers dynamically
tickers = input("Enter tickers separated by commas: ").split(',')
run_analysis([ticker.strip().upper() for ticker in tickers])

Enter tickers separated by commas: hal.ns

Running analysis for HAL.NS...

Results for HAL.NS:
Close Price: 4684.30

Moving Averages:
20-Day MA: 4692.19
50-Day MA: 4994.67
200-Day MA: 3727.66

Fibonacci Retracement Levels:
0.0% (Swing High): 5643.82
23.6%: 4723.79
38.2%: 4154.61
50.0%: 3694.59
61.8%: 3234.58
100.0% (Swing Low): 1745.36

Real-Time Price: 4684.30

Support Levels:
S1: 1745.36
S2: 3234.58
S3: 4154.61

Resistance Levels:
R1: 5643.82
R2: 4154.61
R3: 3234.58
Recommendation: Hold
